# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [2]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [3]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

There might be a problem with your API key? Please visit the troubleshooting notebook!


In [ ]:
# Roche Galileo specific initialization
api_key=os.getenv("OPENAI_API_KEY")
base_url=os.getenv("GALILEO_AI_GATEWAY")

openai = OpenAI(
    api_key=api_key,
    base_url=base_url
)

In [5]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [6]:
ed = Website("https://edwarddonner.com")
ed.links

['https://edwarddonner.com/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/',
 'https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/18/2025-ai-executive-briefing/',
 '

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [7]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}
"""

In [8]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}



In [9]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [10]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://edwarddonner.com - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
https://edwarddonner.com/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/
https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/
https://edward

In [24]:
def get_links(url):
    website = Website(url)
    
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"},
        # timeout=60  # seconds, adjust as needed
    )

    result = response.choices[0].message.content
    return json.loads(result)

In [12]:
# Anthropic has made their site harder to scrape, so I'm using HuggingFace..

huggingface = Website("https://huggingface.co")
huggingface.links

['/',
 '/models',
 '/datasets',
 '/spaces',
 '/docs',
 '/enterprise',
 '/pricing',
 '/login',
 '/join',
 '/spaces',
 '/models',
 '/tencent/SRPO',
 '/Alibaba-NLP/Tongyi-DeepResearch-30B-A3B',
 '/openbmb/VoxCPM-0.5B',
 '/ibm-granite/granite-docling-258M',
 '/google/vaultgemma-1b',
 '/models',
 '/spaces/enzostvs/deepsite',
 '/spaces/zerogpu-aoti/wan2-2-fp8da-aoti-faster',
 '/spaces/Wan-AI/Wan2.2-Animate',
 '/spaces/IndexTeam/IndexTTS-2-Demo',
 '/spaces/abdul9999/NoWatermark',
 '/spaces',
 '/datasets/HuggingFaceFW/finepdfs',
 '/datasets/fka/awesome-chatgpt-prompts',
 '/datasets/InternRobotics/OmniWorld',
 '/datasets/LucasFang/FLUX-Reason-6M',
 '/datasets/HuggingFaceM4/FineVision',
 '/datasets',
 '/join',
 '/pricing#endpoints',
 '/pricing#spaces',
 '/pricing',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/allenai',
 '/facebook',
 '/amazon',
 '/google',
 '/Intel',
 '/microsoft',
 '/grammarly',
 '/Writer',
 '/docs/transforme

In [25]:
get_links("https://huggingface.co")

{'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'},
  {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'},
  {'type': 'blog', 'url': 'https://huggingface.co/blog'},
  {'type': 'models page', 'url': 'https://huggingface.co/models'},
  {'type': 'datasets page', 'url': 'https://huggingface.co/datasets'},
  {'type': 'spaces page', 'url': 'https://huggingface.co/spaces'},
  {'type': 'documentation', 'url': 'https://huggingface.co/docs'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [37]:
from requests.exceptions import SSLError

In [ ]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"

        try:
            result += Website(link["url"]).get_contents()
        except SSLError as e: 
            print(f"Error fetching {link['url']}: {e}")
            
    return result

In [39]:
print(get_all_details("https://huggingface.co"))

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'models page', 'url': 'https://huggingface.co/models'}, {'type': 'datasets page', 'url': 'https://huggingface.co/datasets'}, {'type': 'spaces page', 'url': 'https://huggingface.co/spaces'}, {'type': 'join page', 'url': 'https://huggingface.co/join'}, {'type': 'documentation page', 'url': 'https://huggingface.co/docs'}]}
Error fetching https://apply.workable.com/huggingface/: HTTPSConnectionPool(host='apply.workable.com', port=443): Max retries exceeded with url: /huggingface/ (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self-signed certificate in cert

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


Landing page:
Webpage Title:
Hugging Face – The AI community building the future.
Webpage Contents:
Hugging Face
Models
Datasets
Spaces
Community
Docs
Enterprise
Pricing
Log In
Sign Up
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Explore AI Apps
or
Browse 1M+ models
Trending on
this week
Models
tencent/SRPO
Updated
6 days ago
•
6.62k
•
854
Alibaba-NLP/Tongyi-DeepResearch-30B-A3B
Updated
4 days ago
•
5.04k
•
465
openbmb/VoxCPM-0.5B
Updated
2 days ago
•
1.72k
•
450
ibm-granite/granite-docling-258M
Updated
3 days ago
•
15k
•
402
google/vaultgemma-1b
Updated
9 days ago
•
2.98k
•
354
Browse 1M+ models
Spaces
Running
14k
14k
DeepSite v2
🐳
Generate any application with DeepSeek
Running
on
Zero
MCP
1.17k
1.17k
Wan2.2 14B Fast
🎥
generate a video from an image with a text prompt
Running
155
155
Wan2.2 Animate
👁
Wan2.2 Animate
Running
on
Zero
237
237
IndexTTS 2 Demo
🏢
Generate expressive speech from tex

In [40]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."


In [41]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [ ]:
# Try first to see how the user prompt looks:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog', 'url': 'https://huggingface.co/blog'}, {'type': 'documentation', 'url': 'https://huggingface.co/docs'}]}
Error fetching https://apply.workable.com/huggingface/: HTTPSConnectionPool(host='apply.workable.com', port=443): Max retries exceeded with url: /huggingface/ (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self-signed certificate in certificate chain (_ssl.c:1032)')))


'You are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nHugging Face – The AI community building the future.\nWebpage Contents:\nHugging Face\nModels\nDatasets\nSpaces\nCommunity\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 1M+ models\nTrending on\nthis week\nModels\ntencent/SRPO\nUpdated\n6 days ago\n•\n6.62k\n•\n855\nAlibaba-NLP/Tongyi-DeepResearch-30B-A3B\nUpdated\n4 days ago\n•\n5.04k\n•\n465\nopenbmb/VoxCPM-0.5B\nUpdated\n2 days ago\n•\n1.72k\n•\n450\nibm-granite/granite-docling-258M\nUpdated\n3 days ago\n•\n15k\n•\n402\ngoogle/vaultgemma-1b\nUpdated\n9 days ago\n•\n2.98k\n•\n354\nBrowse 1M+ models\nSpaces\nRunning\n14k\n14k\nDeepSite v2\n🐳\nGener

In [43]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [44]:
create_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'company page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'documentation page', 'url': 'https://huggingface.co/docs'}]}
Error fetching https://apply.workable.com/huggingface/: HTTPSConnectionPool(host='apply.workable.com', port=443): Max retries exceeded with url: /huggingface/ (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self-signed certificate in certificate chain (_ssl.c:1032)')))


# Hugging Face Company Brochure

## **About Us**

Hugging Face is at the forefront of the Artificial Intelligence revolution, fostering a vibrant community focused on building the future of machine learning. Our platform serves as a collaborative space for the machine learning community to create, discover, and share models, datasets, and applications. 

### **Our Vision**
We aim to democratize artificial intelligence, making it accessible to everyone, while accelerating innovation by providing cutting-edge tools and a collaborative environment.

---

## **Products and Services**

### **Collaboration Platform**
- **Models**: Browse over **1 million+ models** designed for various AI tasks.
- **Datasets**: Access **250k+ datasets** for training and evaluation of machine learning models.
- **Spaces**: Create and collaborate on applications in real-time with a community of developers and researchers.

### **Enterprise Solutions**
Hugging Face offers tailored compute and enterprise solutions suited for organizations, ensuring robust security and dedicated support:
- **Pricing**: Starting at $20/user/month.
- **Features**: Includes Single Sign-On, Audit Logs, and Private Dataset Viewer.

### **Open Source Community**
We proudly contribute to the open-source ecosystem, with libraries and tools like Transformers and Diffusers being widely utilized by developers globally.

---

## **Our Customers**
More than **50,000 organizations** leverage Hugging Face for their machine learning needs, including:
- **Meta**
- **Google**
- **Microsoft**
- **Amazon**

---

## **Company Culture**

### **Community-Driven**
At Hugging Face, we value collaboration and open exchange of ideas. Recognizing that the collective strength of our community drives innovation is central to our operations.

### **Supportive Environment**
Our team embodies a culture of inclusivity, aiming to empower every member to contribute meaningfully while continuously learning and growing.

### **Innovative Spirit**
We encourage creativity and embrace challenges as opportunities to innovate and improve our offerings.

---

## **Careers at Hugging Face**

Hugging Face is always looking for passionate individuals eager to make a difference in the AI space. Whether you're a developer, researcher, or marketing expert, we offer diverse career opportunities:

- **Open Positions**: Check out our careers page for the latest job openings in various functions.
- **What We Offer**: Competitive salaries, benefits, and a chance to work alongside industry pioneers in a collaborative environment.

---

## **Get In Touch**
Join us in our mission to build the future of AI! Explore our platform, collaborate with our community, or check out career opportunities at [Hugging Face](https://huggingface.co).

- **Follow us**: GitHub | Twitter | LinkedIn | Discord

---

Hugging Face – The AI community building the future.

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [50]:
# see options : stream=True
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    # Arif: commenting out the print statement below, it was causing issues "IndexError: list index out of range"
    # response = ""
    # display_handle = display(Markdown(""), display_id=True)
    # for chunk in stream:
    #     response += chunk.choices[0].delta.content or ''
    #     response = response.replace("```","").replace("markdown", "")
    #     update_display(Markdown(response), display_id=display_handle.display_id)

    # Arif: replacing the above with the code below to handle possible None values in the stream
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        if chunk is None:
            continue
        elif len(chunk.choices) == 0:
            continue
        elif chunk.choices[0].delta.content is not None:
            response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)


    # Arif: if we don't want Markdown display, just print to console, then it'll be easier
    # for chunk in stream: 
    #     print (chunk.choices[0].delta.content or '', end ='')

In [51]:
stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog', 'url': 'https://huggingface.co/blog'}, {'type': 'company page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}
Error fetching https://apply.workable.com/huggingface/: HTTPSConnectionPool(host='apply.workable.com', port=443): Max retries exceeded with url: /huggingface/ (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self-signed certificate in certificate chain (_ssl.c:1032)')))



# Welcome to the World of Hugging Face! 🌟

## Where AI Hugs You Back!

**Hugging Face** - the AI community that's not just building the future, but also giving it a warm and fuzzy embrace! We are your go-to platform for all things machine learning (ML), where collaboration is the name of the game and AI models are as plentiful as coffee at a tech conference.

### What We Offer 🤖

- **1M+ Models:** That's right! We have so many models, you might feel like you're scrolling through a never-ending cat video feed. From text extraction to video generation, our models are practically in a relationship with your datasets!

- **400k+ Applications:** Need to generate expressive speech from text, or animate a video from an image? Just a few clicks and *voilà*, you’re an AI magicians—or at least you will be after a couple of practice spells. 

- **250k+ Datasets:** Your classic secret sauce for training models! We have mastered the art of dataset collection. Who needs a library card when you have Hugging Face? 

### **The Cool Culture of Hugging Face 🎉**

At Hugging Face, we believe in a culture of camaraderie, co-creation, and a pinch of fun! Our team of 207 members works tirelessly to make ML development a breeze. With a sprinkle of quirkiness, and a dash of passion, we bond over algorithms and dreams of world domination through AI.

- **Work-Life Balance:** Because we don't just work hard, we *play* hard—stable diffusion models by day, dance-offs at night!

- **Diversity & Inclusion:** We celebrate diversity like it's cake day every day! Different experiences and backgrounds? Yes please! It makes our community richer—like training your model on a diverse dataset.

### **Ride the AI Wave with Us! 🌊**

#### **For Prospective Customers:**

Join over 50,000 organizations, including tech giants like Google, Microsoft, and Amazon, who have already made the leap into the future with us! Why should they have all the fun?

#### **For Investors:**

Wondering what to invest in? Forget stocks; invest in crowdsourcing AI talent! Hugging Face is your safety net in a dynamically growing market. Who doesn’t love a high-tech hug with a solid ROI?

#### **Careers at Hugging Face:**

Want to join the Hugging Face team? We are always on the lookout for creative minds who want to push the boundaries of AI. If you can juggle neural networks while riding a unicycle, you’re one of us!

### **Join Us!** 

Sign up today, collaborate with the brightest minds, and get ready to sprinkle some machine learning magic! Because at Hugging Face, we’re not just a community; we’re a family. 🥳

### *Get started on your AI journey with Hugging Face!*

**Visit us at:** [Hugging Face](https://huggingface.co)   
**Follow us on social:** GitHub | Twitter | LinkedIn | Discord

---

*Remember: when in doubt, just add more layers to your neural network!*


In [49]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:
# stream_brochure("HuggingFace", "https://huggingface.co")

system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

create_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'models page', 'url': 'https://huggingface.co/models'}, {'type': 'datasets page', 'url': 'https://huggingface.co/datasets'}, {'type': 'spaces page', 'url': 'https://huggingface.co/spaces'}]}
Error fetching https://apply.workable.com/huggingface/: HTTPSConnectionPool(host='apply.workable.com', port=443): Max retries exceeded with url: /huggingface/ (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self-signed certificate in certificate chain (_ssl.c:1032)')))


# Welcome to the World of Hugging Face!

### **Where AI & Machine Learning Meet with a Big Virtual Hug!** 🤗

---

#### **Who Are We?**

We’re Hugging Face, your friendly neighborhood AI community! We’re building the future one model, dataset, and collaboration at a time, all while ensuring everyone is adequately hugged—virtually, of course! 

Over 50,000 organizations, from small startups to tech giants like Google and Amazon, are using our platform to create a world where technology understands you better than your best friend ever could.

---

#### **What We Do?**

At Hugging Face, we believe in community collaboration. That means we host and share:

- **1M+ Models**: Yes, you read that right! There's probably a model for **everything**. We have more models than your average magic store has tricks up its sleeves!
  
- **250K+ Datasets**: With this much data, we could probably solve world hunger if only we could find a model for that... 

- **400K+ Applications**: Why not dive into our rich ocean of applications that let you generate speech, create videos from images, and even remove watermarks (don’t worry, we won’t tell your secrets)!

---

#### **Join Our Hug-tastic Culture** 

- **We're Open Source!** 💻: We believe in sharing and caring. With tools like Transformers and Tokenizers, you can turn your machine learning dreams into a reality! 
- **Work Hard, Hug Harder**: We value collaboration so much, we might even hug it out after our next team meeting. 
- **Continuous Learning**: Keep an eye on our blog and forum for the latest tips, tricks, and hilarious AI bloopers!

---

#### **Careers at Hugging Face!** 🚀

- **Job Openings**: If you’re passionate about machine learning, come join us! You might find yourself building the next big model or updating some datasets right in the middle of an IT hip-hop revolution. 
- **Team Spirit**: We don’t just work; we play. Dunking on each other in ping pong and sharing memes are essential team bonding exercises. 
- **Grow With Us**: Not only will you advance your career, but you might also find your AI soulmate! 

---

#### **Our Dearest Customers** ❤️

From nonprofits to well-known tech giants like Microsoft and Grammarly, we proudly serve customers from all walks of life. Together, we are shaping the future of AI and Machine Learning!

---

### **In Summary...** 

At Hugging Face, we’re more than just a company; we’re a family of AI enthusiasts who believe in collaboration, innovation, and the occasional hug! Join us on this wildly exciting adventure, and who knows? You might just invent that model that finally understands your dad jokes!

---

**Interested?** Ready to embrace the future with us? Check out our website at [HuggingFace.co](https://huggingface.co) and bring a hug or two! 🤗❤️

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype. See what other students have done in the community-contributions folder -- so many valuable projects -- it's wild!</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 3 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!<br/>
            3. I'm trying out X/Twitter and I'm at <a href="https://x.com/edwarddonner">@edwarddonner<a> and hoping people will teach me how it's done..  
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>